[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Qiuyi-Hong/SHEMS/blob/main/analysisSHEMS_v4.ipynb)

In [1]:
# # Install necessary packages: 
# !pip install pyomo pandas openpyxl matplotlib jupyter

# # Install cbc solver: 
# !sudo apt-get install  coinor-cbc coinor-libcbc-dev 

# # Download necessary files:
# import urllib.request, sys

# urls = [
#     'https://raw.githubusercontent.com/Qiuyi-Hong/SHEMS/refs/heads/main/SHEMS_v3.py',
#     'https://raw.githubusercontent.com/Qiuyi-Hong/SHEMS/refs/heads/main/SHEMS_v3.dat',
#     'https://raw.githubusercontent.com/Qiuyi-Hong/SHEMS/refs/heads/main/Simulation%20Results/simulation_results_concatenated.csv',
#     'https://raw.githubusercontent.com/Qiuyi-Hong/SHEMS/refs/heads/main/agile-half-hour-actual-rates-25-01-2023_26-01-2024.csv',
#     'https://raw.githubusercontent.com/Qiuyi-Hong/SHEMS/refs/heads/main/simulation_results_completed.csv',
#     ]
# file_names = [url.split('/')[-1] for url in urls]

# for i in range(len(urls)):
#     urllib.request.urlretrieve(urls[i], file_names[i])


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from datetime import datetime, timedelta
%matplotlib inline
import pyomo.environ as pyo
from shems import shems_model

# Data Preparation

In [3]:
data = pd.read_csv("simulation_results_finished.csv")
data.head()

,Date,Dry-bulb temperature (°C),Total system energy (kBtu/h),System natural gas (kBtu/h),System electricity (kBtu/h),Total lights energy (kBtu/h),Lights electricity (kBtu/h),Total equip energy (kBtu/h),Equip electricity (kBtu/h),Total CE ex equip (kgCO2/h),...,Total electricity (kBtu/h),Total grid displaced elec. PV (kBtu/h),Total energy (kBtu/h),Transformer Losses (kBtu/h),Agile Import price (p/kWh),Agile Export price (p/kWh),DHW demand (kW),SH demand (kW),Total heat demand (kW),Ele demand (kW)
0,2023-01-01 00:00:00,5.5,7.460,7.325,0.135,0.0,0.0,2.912,2.912,0.5,...,3.048,0.0,10.372,0.0,4.41,2.77,0.274968,2.086286,2.361254,0.228635
1,2023-01-01 00:30:00,5.5,7.465,7.330,0.135,0.0,0.0,2.912,2.912,0.5,...,3.048,0.0,10.378,0.0,13.91,6.86,0.275170,2.086286,2.361455,0.228635
2,2023-01-01 01:00:00,5.5,7.468,7.333,0.135,0.0,0.0,2.912,2.912,0.5,...,3.048,0.0,10.380,0.0,15.44,7.52,0.275270,2.108282,2.383553,0.228635
3,2023-01-01 01:30:00,5.5,7.467,7.331,0.135,0.0,0.0,2.912,2.912,0.5,...,3.048,0.0,10.379,0.0,1.10,1.34,0.275213,2.108282,2.383495,0.228635
4,2023-01-01 02:00:00,5.5,7.465,7.330,0.135,0.0,0.0,2.912,2.912,0.5,...,3.048,0.0,10.378,0.0,6.70,3.76,0.275184,2.106590,2.381774,0.228635


In [4]:
data.set_index('Date', inplace=True)
# Slice the DataFrame from a start date to the end
start_date = '2023-11-01 00:00:00'
end_date = '2023-11-30 23:30:00'
data = data.loc[start_date:end_date]
data.reset_index(inplace=True)
T = data.shape[0]

# Convert the format of the start date for the figure file name
modified_start_date = start_date.replace(':', '_').replace(' ', '_')

In [5]:
# Solar PV generation
eta = 0.157
S = 20
solar_radiation = data["Direct radiation (W/m²)"]/1000
outdoor_temp = data["Dry-bulb temperature (°C)"]

pv = eta * S * solar_radiation *(1 - 0.005 * (outdoor_temp - 25))

In [6]:
# COP calculation
a = - 0.087
b = 6.8 
COP = a * (outdoor_temp + 30 - outdoor_temp) + b 

In [7]:
delta_t = 0.5

demand_ele = data["Ele demand (kW)"]
demand_gas = data["Total heat demand (kW)"]/0.95

demand_SH = data["SH demand (kW)"]
demand_DHW = data["DHW demand (kW)"]
demand_heat = data["Total heat demand (kW)"][:T]

# Electricity price
ele_import_price = data["Agile Import price (p/kWh)"]
ele_export_price = data["Agile Export price (p/kWh)"]

In [8]:
# Accurate Q_TES_init calculation
T_TES_init = 50
T_inlet = 10
V_TES = 0.3

Q_TES_init = (1000*V_TES*4186*(T_TES_init - T_inlet))/3.6e6
print(Q_TES_init)

# Accurate Q_TES_max calculation
T_TES_max = 70
Q_TES_max = (1000*V_TES*4186*(T_TES_max - T_inlet))/3.6e6
print(Q_TES_max)

13.953333333333333
20.93


In [9]:
inputs = {
    "T": T,
    "delta_t": delta_t,
    "d_ele": demand_ele,
    "d_SH": demand_SH,
    "d_DHW": demand_DHW,
    "pi_import": ele_import_price,
    "pi_export": ele_export_price,
    # ASHP
    "T_out": outdoor_temp,
    "COP": COP,
    "q_HP_max": 10,
    "q_HP_min": 0,
    # Comfort params
    ## For SH
    "rho_in": 1.2041,
    "V_in": 3277,
    "c_in": 1000,
    "T_in_LB": 19,
    "T_in_UB": 23,
    "K_SH": 0.0025,
    "T_in_init": 21,
    ## For DHW
    "c_TES": 4186,
    "T_TES_LB": 45,
    "T_TES_UB": 55,
    "K_TES": 0.0025,
    "T_TES_init": T_TES_init,
    # TES params
    "Q_TES_min": 0,
    "Q_TES_max": Q_TES_max,
    "Q_TES_init": Q_TES_init,
    "V_TES": V_TES,
    "rho_TES": 1000,
    "T_inlet": T_inlet,
    "T_TES_max": T_TES_max,
    "p_pv": pv
}

# SHEMS

In [10]:
shems = shems_model(inputs)

In [11]:
from sys import platform

if platform == "linux":
    # linux
    solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
elif platform == "darwin":
    # maxOS
    solver = pyo.SolverFactory('cplex', executable = '/Applications/CPLEX_Studio221/cplex/bin/x86-64_osx/cplex')
    # solver = pyo.SolverFactory('scip', solver_io='nl')
    # solver = pyo.SolverFactory('cbc')
    solver = pyo.SolverFactory('gurobi')

solver.solve(shems)

{'Problem': [{'Name': 'x1', 'Lower bound': 8708.399550449143, 'Upper bound': 8709.264149871053, 'Number of objectives': 1, 'Number of constraints': 33120, 'Number of variables': 18720, 'Number of binary variables': 2880, 'Number of integer variables': 2880, 'Number of continuous variables': 15840, 'Number of nonzeros': 61917, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '2.5822839736938477', 'Error rc': 0, 'Time': 2.7658140659332275}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
obj_value = pyo.value(shems.obj)
print("Objective function value:", obj_value)

Objective function value: 8709.264149871045


# Gas Boiler Comparison

In [13]:
Jan_Mar_gas_cap = 2137/12000 # 0.17808333
Apr_Jun_gas_cap = 1610/12000 # 0.13416667
Jul_Sep_gas_cap = 998/12000 # 0.08316667
Oct_Dec_gas_cap = 926/12000 # 0.07716667

avg_gas_cap = (Jan_Mar_gas_cap + Apr_Jun_gas_cap + Jul_Sep_gas_cap + Oct_Dec_gas_cap)/4
avg_gas_cap # = 0.11814583333333334

annual_avg_gas_price = 1304/13600 # 0.09588235

rate = annual_avg_gas_price/avg_gas_cap # 0.8100000000000001

Jan_Mar_gas_avg = Jan_Mar_gas_cap * rate * 100 # 14.42474973
Apr_Jun_gas_avg = Apr_Jun_gas_cap * rate * 100 # 10.8485623
Jul_Sep_gas_avg = Jul_Sep_gas_cap * rate * 100 # 6.7323747
Oct_Dec_gas_avg = Oct_Dec_gas_cap * rate * 100 # 6.2323747

In [14]:
# Assuming 'data' is your DataFrame and 'Date' is already in datetime format
data['Date'] = pd.to_datetime(data['Date'])

costs_total_gas = 0

for index, row in data.iterrows():
    t = row['Date']
    if t >= pd.to_datetime('2023-01-01 00:00:00') and t <= pd.to_datetime('2023-03-31 23:30:00'):
        gas_price = Jan_Mar_gas_avg     
        if demand_ele[index] - pv[index] >= 0:
            costs_total_gas += demand_gas[index] * delta_t * gas_price + ele_import_price[index] * (demand_ele[index] - pv[index]) * delta_t
        else:
            costs_total_gas += demand_gas[index] * delta_t * gas_price - ele_export_price[index] * (pv[index] - demand_ele[index]) * delta_t
    elif t >= pd.to_datetime('2023-04-01 00:00:00') and t <= pd.to_datetime('2023-06-30 23:30:00'):
        gas_price = Apr_Jun_gas_avg
        if demand_ele[index] - pv[index] >= 0:
            costs_total_gas += demand_gas[index] * delta_t * gas_price + ele_import_price[index] * (demand_ele[index] - pv[index]) * delta_t
        else:
            costs_total_gas += demand_gas[index] * delta_t * gas_price - ele_export_price[index] * (pv[index] - demand_ele[index]) * delta_t
    elif t >= pd.to_datetime('2023-07-01 00:00:00') and t <= pd.to_datetime('2023-09-30 23:30:00'):
        gas_price = Jul_Sep_gas_avg
        if demand_ele[index] - pv[index] >= 0:
            costs_total_gas += demand_gas[index] * delta_t * gas_price + ele_import_price[index] * (demand_ele[index] - pv[index]) * delta_t
        else:
            costs_total_gas += demand_gas[index] * delta_t * gas_price - ele_export_price[index] * (pv[index] - demand_ele[index]) * delta_t
    elif t >= pd.to_datetime('2023-10-01 00:00:00') and t <= pd.to_datetime('2023-12-31 23:30:00'):
        gas_price = Oct_Dec_gas_avg
        if demand_ele[index] - pv[index] >= 0:
            costs_total_gas += demand_gas[index] * delta_t * gas_price + ele_import_price[index] * (demand_ele[index] - pv[index]) * delta_t
        else:
            costs_total_gas += demand_gas[index] * delta_t * gas_price - ele_export_price[index] * (pv[index] - demand_ele[index]) * delta_t

In [15]:
costs_total_gas

np.float64(12143.794542268968)

In [16]:
# Create a file and write the results to it
with open('cost_results_monthly.txt', 'a') as file:
    file.write(f"From {start_date} to {end_date}: Cost HP: {pyo.value(shems.obj)}, Cost Gas Boiler: {costs_total_gas}\n")

In [17]:
# # This cell needs to be run only once to create the initial CSV file!!!
# # Create initial DataFrame
# data = {
#     'From': [],
#     'To': [],
#     'Cost_HP': [],
#     'Cost_Gas_Boiler': []
# }
# df = pd.DataFrame(data)

# # Write to CSV file
# df.to_csv('cost_results_monthly.csv', index=False)

In [18]:
# Read the existing CSV file
existing_df = pd.read_csv('cost_results_monthly.csv')

# Create new data to append
new_data = {
    'From': [start_date],
    'To': [end_date],
    'Cost_HP': [pyo.value(shems.obj)],
    'Cost_Gas_Boiler': [costs_total_gas]
}
new_df = pd.DataFrame(new_data)

# Append new data to the existing DataFrame
updated_df = pd.concat([existing_df, new_df], ignore_index=True)

In [19]:
# Write the updated DataFrame back to the CSV file
updated_df.to_csv('cost_results_monthly.csv', index=False)